In [1]:
import tensorflow as tf
import time
import os
print(tf.__version__)
data_dir = './datasets'
train_cats_dir = data_dir + '/train/cats/'
train_dogs_dir = data_dir + '/train/dogs/'
test_cats_dir = data_dir + '/valid/cats/'
test_dogs_dir = data_dir + '/valid/dogs/'


# 构建训练数据集
train_cat_filenames = tf.constant([train_cats_dir + filename for filename in os.listdir(train_cats_dir)][:1000])
train_dog_filenames = tf.constant([train_dogs_dir + filename for filename in os.listdir(train_dogs_dir)][:1000])
train_filenames = tf.concat([train_cat_filenames, train_dog_filenames], axis=-1)
train_labels = tf.concat([
    tf.zeros(train_cat_filenames.shape, dtype=tf.int32), 
    tf.ones(train_dog_filenames.shape, dtype=tf.int32)], 
    axis=-1)

2.0.0


In [2]:
#构建训练集

def _decode_and_resize(filename, label):
    image_string = tf.io.read_file(filename)            # 读取原始文件
    image_decoded = tf.image.decode_jpeg(image_string)  # 解码JPEG图片
    image_resized = tf.image.resize(image_decoded, [256, 256]) / 255.0
    return image_resized, label


batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((train_filenames, train_labels))

def benchmark(dataset, num_epochs=1):
    start_time = time.perf_counter()
    for epoch_num in range(num_epochs):
        for sample in dataset:
            # Performing a training step
            time.sleep(0.01)
    tf.print("Execution time:", time.perf_counter() - start_time)

In [3]:
1

1

# prefetch方法：

In [4]:
benchmark(train_dataset.map(
    map_func=_decode_and_resize, 
    num_parallel_calls=tf.data.experimental.AUTOTUNE),
    num_epochs=1)

Execution time: 29.59943906086638


In [5]:
benchmark(
    train_dataset.map(
    map_func=_decode_and_resize, 
    num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .prefetch(tf.data.experimental.AUTOTUNE),
    num_epochs=1
)

Execution time: 29.054936807737768


# interleave 方法

In [6]:
filenames = ["./interleave_data/train.csv", "./interleave_data/eval.csv",
             "./interleave_data/train.csv", "./interleave_data/eval.csv",]
dataset = tf.data.Dataset.from_tensor_slices(filenames)

def data_func(line):
    line = tf.strings.split(line, sep = ",")
    return line

dataset_1 = dataset.interleave(lambda x:
    tf.data.TextLineDataset(x).skip(1).map(data_func),
    cycle_length=4, block_length=16)

In [7]:
for line in dataset_1.take(2):
    print(line)

tf.Tensor(
[b'0' b'male' b'22.0' b'1' b'0' b'7.25' b'Third' b'unknown' b'Southampton'
 b'n'], shape=(10,), dtype=string)
tf.Tensor(
[b'1' b'female' b'38.0' b'1' b'0' b'71.2833' b'First' b'C' b'Cherbourg'
 b'n'], shape=(10,), dtype=string)


In [8]:
benchmark(dataset_1,
    num_epochs=1)

Execution time: 25.970601931961596


In [9]:
dataset_2 = dataset.interleave(lambda x:
    tf.data.TextLineDataset(x).skip(1).map(data_func),
    num_parallel_calls=tf.data.experimental.AUTOTUNE,
    cycle_length=4, block_length=16)

In [10]:
benchmark(dataset_2,
    num_epochs=1)

Execution time: 25.826337882865033


# map方法

In [11]:
benchmark(
    train_dataset.map(
    map_func=_decode_and_resize, 
    num_parallel_calls=tf.data.experimental.AUTOTUNE)
)

Execution time: 29.13341431492455


In [12]:
benchmark(
    train_dataset.map(
    map_func=_decode_and_resize
    )
)

Execution time: 28.908936553196412


# cache方法 

In [3]:
benchmark(
    train_dataset.map(
    map_func=_decode_and_resize,
    num_parallel_calls=tf.data.experimental.AUTOTUNE
    ),
    num_epochs=2
)

Execution time: 57.88074226885329


In [4]:
benchmark(
    train_dataset.map(
    map_func=_decode_and_resize,
    num_parallel_calls=tf.data.experimental.AUTOTUNE
    ).cache(),
    num_epochs=2
)

Execution time: 57.8187796820567
